# Notebook to generate psychometrics plot given sessions & conditions

In [1]:
import datajoint as dj
import pandas as pd
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches

from matplotlib import cm
from u19_pipeline import utility
from inspect import getmembers, isfunction
from datetime import datetime
import u19_pipeline.utils.slack_utils as su
import time


### Datajoint configuration and Connection to DB

In [2]:
utility.basic_dj_configuration(dj)
dj.conn()

[2025-05-07 21:53:19,490][INFO]: Connecting alvaros@datajoint00.pni.princeton.edu:3306
[2025-05-07 21:53:21,238][INFO]: Connected alvaros@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) alvaros@datajoint00.pni.princeton.edu:3306

## Databases to connect

In [3]:
acquisition = dj.create_virtual_module('acquisition', 'u19_acquisition')
lab = dj.create_virtual_module('lab', 'u19_lab')
subject = dj.create_virtual_module('subject', 'u19_subject')

In [4]:
slack_configuration_dictionary = {
    'slack_notification_channel': ['alvaro_luna']
}
webhooks_list = []
query_slack_webhooks = [{'webhook_name' : x} for x in slack_configuration_dictionary['slack_notification_channel']]
webhooks_list += (lab.SlackWebhooks & query_slack_webhooks).fetch('webhook_url').tolist()

In [5]:
def slack_alert_message_format_live_stats(alert_dictionary1, alert_dictionary2, time_no_response):

    now = datetime.now()
    datestr = now.strftime('%d-%b-%Y %H:%M:%S')

    msep = dict()
    msep['type'] = "divider"

    #Title#
    m1 = dict()
    m1['type'] = 'section'
    m1_1 = dict()
    m1_1["type"] = "mrkdwn"
    m1_1["text"] = ':rotating_light: * Live Monitor Alert* on ' + datestr + '\n' +\
    'More than ' + str(time_no_response) + ' s without new trial' + '\n'
    m1['text'] = m1_1

    #Info#
    m2 = dict()
    m2['type'] = 'section'
    m2_1 = dict()
    m2_1["type"] = "mrkdwn"

    m2_1["text"] = '*Session Reported:*' + '\n'
    for key in alert_dictionary1.keys():
        m2_1["text"] += '*' + key + '* : ' + str(alert_dictionary1[key]) + '\n'
    m2_1["text"] += '\n'
    m2['text'] = m2_1

    m4 = dict()
    m4['type'] = 'section'
    m4_1 = dict()
    m4_1["type"] = "mrkdwn"

    m4_1["text"] = '*Last Stats Reported*:' + '\n'
    for key in alert_dictionary2.keys():
        m4_1["text"] += '*' + key + '* : ' + str(alert_dictionary2[key]) + '\n'
    m4_1["text"] += '\n'
    m4['text'] = m4_1

    message = dict()
    message['blocks'] = [m1,msep,m2,msep,m4,msep]
    message['text'] = 'Live Monitor Alert'

    return message

In [ ]:
# Query today's started sessions that are not finished
query = {}
query['session_date'] = datetime.today().strftime('%Y-%m-%d')
query['is_finished'] = 0

sessions = pd.DataFrame((acquisition.SessionStarted & query).fetch('KEY',as_dict=True))
sessions = sessions.loc[~sessions['subject_fullname'].str.startswith('testuser'),:]

# Query last live stat from the started sessions
query_live_stats = sessions.to_dict('records')
lss = acquisition.SessionStarted.aggr(acquisition.LiveSessionStats.proj('current_datetime'), current_datetime="max(current_datetime)")
live_stats = pd.DataFrame((lss & query_live_stats).fetch(as_dict=True))

# If there are any sessions with live stats
if live_stats.shape[0] > 0:

    # Filter sessions whose last trial info is greater than 300s
    live_stats['last_time'] = (datetime.now()- live_stats['current_datetime']).dt.total_seconds()
    live_stats['alert'] = live_stats['last_time'] >300
    live_stats = live_stats.loc[live_stats['alert']==True,:]

    #If there are any sessions to alert (more then 300s)
    if live_stats.shape[0] > 0:

        #get_session_info to alert (plus slack researcher)
        query_live_stats_sessions = live_stats[['subject_fullname', 'session_date', 'session_number']].to_dict('records')

        session_data_df = pd.DataFrame(((lab.User.proj('slack') * subject.Subject.proj('user_id') *\
                                    acquisition.SessionStarted.proj('session_location')) & query_live_stats_sessions).fetch(as_dict=True))
        
        session_data_df = session_data_df.rename({'slack': 'researcher'}, axis=1)
        session_data_df['researcher'] = '<@'+ session_data_df['researcher'] + '>'
        session_data_df = session_data_df[['researcher', 'subject_fullname', 'session_date', 'session_number']]

        #Query full live stat table
        session_stats = live_stats.copy()
        session_stats = session_stats.rename({'current_datetime': 'last_live_stat'}, axis=1)
        query_live_stats = live_stats[['subject_fullname', 'session_date', 'session_number', 'current_datetime']].to_dict('records')
        live_stats = live_stats.drop(columns=['current_datetime'])
        ls_full_df = pd.DataFrame((acquisition.LiveSessionStats & query_live_stats).fetch(as_dict=True))
        ls_full_df = pd.merge(ls_full_df, live_stats, on=['subject_fullname', 'session_date', 'session_number'])
        ls_full_df = ls_full_df.drop(columns=['subject_fullname', 'session_date', 'session_number'])
        ls_full_df = ls_full_df.rename({'current_dateime': 'last_stat_time'}, axis=1)

        mid = ls_full_df['last_stat_time']
        ls_full_df = ls_full_df.drop(columns=['last_stat_time'])
        ls_full_df.insert(0, 'last_stat_time', mid)

        ls_full_dict = ls_full_df.to_dict('records')

        # Send one alert per session found
        idx_alert = 0
        for this_alert_record in ls_full_dict:

            #Format message for session and live stat dictionary
            this_session_stats = session_data_df.iloc[idx_alert,:]
            slack_json_message = slack_alert_message_format_live_stats(this_session_stats.to_dict(), this_alert_record, int(this_alert_record['last_time']))

            #Send alert
            for this_webhook in webhooks_list:
                su.send_slack_notification(this_webhook, slack_json_message)
                time.sleep(1)

            idx_alert += 1

    


{'Content-Type': 'application/json', 'Content-Length': '184'}
{'blocks': [{'type': 'section', 'text': {'type': 'mrkdwn', 'text': ':rotating_light: * Live Monitor Alert* on 07-May-2025 21:59:21\nMore than 38371 s without new trial\n'}}, {'type': 'divider'}, {'type': 'section', 'text': {'type': 'mrkdwn', 'text': '*Session Reported:*\n*researcher* : <@UR05564BH>\n*subject_fullname* : efonseca_ef559_act121\n*session_date* : 2025-05-07\n*session_number* : 0\n\n'}}, {'type': 'divider'}, {'type': 'section', 'text': {'type': 'mrkdwn', 'text': '*Last Stats Reported*:\n*block* : 1\n*trial_idx* : 278\n*current_datetime* : 2025-05-07 11:19:50\n*total_trials* : 276\n*level* : 3\n*sublevel* : 1\n*num_trials_left* : 133\n*num_trials_right* : 143\n*performance* : 1.0\n*performance_right* : 1.0\n*performance_left* : 1.0\n*bias* : -0.00052579\n*mean_duration_trial* : 10.2853\n*median_duration_trial* : 7.31453\n*last_time* : 38371.238824\n*alert* : True\n\n'}}, {'type': 'divider'}], 'text': 'Live Monitor